In [15]:
CHARACTER = "ФИБИ"

main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']
labels_to_chars = dict(enumerate(main_characters))
chars_to_labels = {v: k for k, v in labels_to_chars.items()}
labels_to_chars, chars_to_labels

({0: 'ДЖОУИ', 1: 'МОНИКА', 2: 'РЕЙЧЕЛ', 3: 'РОСС', 4: 'ФИБИ', 5: 'ЧЕНДЛЕР'},
 {'ДЖОУИ': 0, 'МОНИКА': 1, 'РЕЙЧЕЛ': 2, 'РОСС': 3, 'ФИБИ': 4, 'ЧЕНДЛЕР': 5})

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!git clone https://github.com/Alenush/style_transfer_sirius2021summer.git

Cloning into 'style_transfer_sirius2021summer'...
remote: Enumerating objects: 1679, done.
remote: Counting objects: 100% (1679/1679), done.
remote: Compressing objects: 100% (1035/1035), done.
remote: Total 1679 (delta 753), reused 1538 (delta 617), pack-reused 0
Receiving objects: 100% (1679/1679), 44.99 MiB | 10.94 MiB/s, done.
Resolving deltas: 100% (753/753), done.


In [29]:
%%bash

cd style_transfer_sirius2021summer
git checkout master
git pull
cd -

Branch 'master' set up to track remote branch 'master' from 'origin'.
Already up to date.
/content


Switched to a new branch 'master'


In [17]:
chars_ru2en = {
    'ДЖОУИ'  : 'Joey',
    'МОНИКА' : 'Monica',
    'РЕЙЧЕЛ' : 'Rachel',
    'РОСС'   : 'Ross',
    'ФИБИ'   : 'Phoebe',
    'ЧЕНДЛЕР': 'Chandler',
}
chars_ru2en

{'ДЖОУИ': 'Joey',
 'МОНИКА': 'Monica',
 'РЕЙЧЕЛ': 'Rachel',
 'РОСС': 'Ross',
 'ФИБИ': 'Phoebe',
 'ЧЕНДЛЕР': 'Chandler'}

In [18]:
#!g1.1
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [24]:
import pathlib

# The name of the task to train.I'm going to name this 'yelp'.
EN_CHAR_NAME = chars_ru2en[CHARACTER].lower()
TASK_NAME = f'en-{EN_CHAR_NAME}'

ROOT_DIR = '/content/drive/MyDrive/FriendsClassifiers'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'{ROOT_DIR}/outputs/{TASK_NAME}/'

WEIGHTS_NAME = f"{TASK_NAME}-pytorch_model.bin"
NORMAL_WEIGHTS_NAME = "pytorch_model.bin"

In [20]:
OUTPUT_DIR

'/content/drive/MyDrive/FriendsClassifiers/outputs/en-phoebe/'

In [30]:
import os

os.rename(f'{OUTPUT_DIR}{WEIGHTS_NAME}', f'{OUTPUT_DIR}{NORMAL_WEIGHTS_NAME}')

FileNotFoundError: ignored

In [ ]:
!pip install transformers

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(OUTPUT_DIR)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR)

# Copy the model to the GPU.
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [26]:
!pip install rusenttokenize

In [31]:
from rusenttokenize import ru_sent_tokenize
filename = f"./style_transfer_sirius2021summer/data/generated/{EN_CHAR_NAME}.txt"
with open(filename) as f:
    phoebe_replics = f.readlines()

phoebe_replics = [replic.strip() for replic in phoebe_replics]

phoebe_evaluation_dict = dict()
for replic in phoebe_replics:
    for sentence in ru_sent_tokenize(replic):
        phoebe_evaluation_dict[sentence] = -1

In [ ]:
import pandas as pd

df = pd.read_csv("unique_phrases_by_character.csv").iloc[:, 1:]
df

In [34]:
import pandas as pd

df_eval = pd.DataFrame(columns=["sentence", 'label'])
df_eval['sentence'] = phoebe_evaluation_dict.keys()
df_eval["sentence"] = df_eval["sentence"].apply(lambda x: x.strip())
df_eval["label"] = 1
df_eval = pd.concat([df_eval, df_phoebe[['label','sentence']].sample(n=200, random_state=1)]).reset_index(drop=True)
df_eval = df_eval.sample(frac=1)
df_eval.shape

NameError: ignored

In [32]:
import pandas as pd

# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df_eval.shape[0]))

# Create sentence and label lists
sentences = df_eval.sentence.values
labels = df_eval.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

NameError: ignored

In [ ]:
df_eval